In [ ]:
# This program get additional information for the process "Regimen Especial" and "Subasta Inversa Electrónica" which status is "Adjudicado" 
# To avoid getting error during the process, I run this program per each year since 2017 to 2020

import os
import glob
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

path = 'D:/hagp/Data/'

    
def getDataContratos(dfContrato,year):
     
     firstTime = 0
     df = pd.DataFrame(columns=['Code', 'Budget','Ruc','Company','Execution-Date','TotalCost'])

     
     for index, row in dfContrato.iterrows():
          url = row["Link"]
          codeProcess = row["Code"]
          my_driver.get(url)  # Abrir una nueva instancia del browser
          
          if(firstTime==0):
               my_driver.maximize_window()  
               my_driver.find_element_by_xpath("//button[@class='cc-btn cc-dismiss']").click() 
                                             
          time.sleep(4)
          print(codeProcess)
          if(codeProcess.startswith('RE')):
               my_driver.find_element_by_xpath("//ul[@id='menu']//li//ul//li//a").click()
          else:
               my_driver.find_element_by_xpath("//ul[2]//li[1]//ul[1]//li[1]//a[1]").click()
                                                  
               

          time.sleep(8)
          code = my_driver.find_element_by_xpath("//body/div[@id='contenedor']/div/div/form[@id='frminfoGeneralContrato']/table/tbody/tr[3]/td[3]").text
                                                  
          budget = my_driver.find_element_by_xpath("//body/div[@id='contenedor']/div/div/form[@id='frminfoGeneralContrato']/table/tbody/tr[7]/td[3]").text
          ruc= my_driver.find_element_by_xpath("//tr[15]//td[1]//table[1]//tbody[1]//tr[2]//td[2]").text
          company = my_driver.find_element_by_xpath("//tr[15]//td[1]//table[1]//tbody[1]//tr[2]//td[3]").text
          date = my_driver.find_element_by_xpath("//tr[15]//td[1]//table[1]//tbody[1]//tr[2]//td[4]").text
          totalCost = my_driver.find_element_by_xpath("//tr[15]//td[1]//table[1]//tbody[1]//tr[2]//td[5]").text
          time.sleep(5)
          budget = budget.replace("$","").replace(",","")
          totalCost = totalCost.replace(",","").strip()
          values_to_add = {'Code': code.strip(), 'Budget': budget.strip(), 'Ruc':str(ruc).strip(), 'Company':company.strip(),'Execution-Date':date,'TotalCost':totalCost}
          
          row_to_add = pd.Series(values_to_add, name = firstTime+1)
          df = df.append(row_to_add)
          
          time.sleep(2)
          firstTime+=1
          
         
     df.to_csv("D:/hagp/data-contratos/" + year + "-additionaldata.csv")
     my_driver.close()
     mergeDf(dfContrato,df,year)
    
def mergeDf(dfContratos, dfAdicional, year):
    
     columns = dfContratos.columns 
     mergedDf = dfContratos.merge(dfAdicional, how='left', left_on='Code', right_on='Code')
     
     mergedDf.to_csv("D:/hagp/data-final/" + year + "-informacion-contratos.csv")

def concatCSV(path, year):
     #year = "2019"
     #os.chdir("D:/hagp/Data/"+year)
     dirFile = path + year
     os.chdir(dirFile)
     extension = 'csv'
     all_filenames = [i for i in (glob.glob('*.{}'.format(extension)))]
    
     #combine all files in the list
     combinedCsv = pd.concat([pd.read_csv(f) for f in all_filenames ])
     columns = combinedCsv.columns
     combinedCsv = combinedCsv.drop([columns[0]], axis=1)
    
     columns2 = combinedCsv.columns #consultar las columnas nuevamente para reemplazar los status
     combinedCsv[columns2[2]] = combinedCsv[columns2[2]].apply(lambda x : x.replace("Ejecución de Contrato", "Adjudicado").replace("Finalizada", "Adjudicado").replace("En Recepción","Adjudicado"))
     
     #seleccionar los registros con status Adjudicado     
     dfContratos = combinedCsv[combinedCsv['Status'] == "Adjudicado"]
     
     getDataContratos(dfContratos, year)






options = Options()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument("--headless")
my_driver = webdriver.Chrome(options=options,executable_path=r'D:\webdrivers\chromedriver.exe')

concatCSV(path,"2017")